In [68]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [69]:
m2018 = pd.read_csv('data/matriculas/2018/m2018.csv', sep=';')
m2019 = pd.read_csv('data/matriculas/2019/m2019.csv', sep=';')

/var/folders/v5/8__wxdkj3j713bygpr1tsw_80000gn/T/ipykernel_88741/3969867569.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  m2018 = pd.read_csv('data/matriculas/2018/m2018.csv', sep=';')
/var/folders/v5/8__wxdkj3j713bygpr1tsw_80000gn/T/ipykernel_88741/3969867569.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  m2019 = pd.read_csv('data/matriculas/2019/m2019.csv', sep=';')


In [70]:
from analisis import estudiantes

In [71]:
estudiantes2018 = estudiantes(m2018, m2019)

/Users/davidlealolivares/Library/Mobile Documents/com~apple~CloudDocs/Talleres y ayudantias Diplomado DataScience UC2023/desercion-escolar/analisis.py:123: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  desertores = desertores[~((desertores['COD_GRADO'] == 4) & (df_anterior['COD_ENSE2'].isin([5, 7])))]


In [72]:
# Importar estudiantes 2018
estudiantes2018 = pd.read_csv('data/tablas/estudiantes2018.csv', sep=';')

In [73]:
# info
estudiantes2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79503 entries, 0 to 79502
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      79503 non-null  int64 
 1   AGNO            79503 non-null  int64 
 2   RBD             79503 non-null  int64 
 3   DGV_RBD         79503 non-null  int64 
 4   NOM_RBD         79503 non-null  object
 5   COD_REG_RBD     79503 non-null  int64 
 6   COD_PRO_RBD     79503 non-null  int64 
 7   COD_COM_RBD     79503 non-null  int64 
 8   NOM_COM_RBD     79503 non-null  object
 9   COD_DEPROV_RBD  79503 non-null  int64 
 10  NOM_DEPROV_RBD  79503 non-null  object
 11  COD_DEPE        79503 non-null  int64 
 12  COD_DEPE2       79503 non-null  int64 
 13  RURAL_RBD       79503 non-null  int64 
 14  ESTADO_ESTAB    79503 non-null  int64 
 15  COD_ENSE        79503 non-null  int64 
 16  COD_ENSE2       79503 non-null  int64 
 17  COD_ENSE3       79503 non-null  int64 
 18  COD_GR

In [74]:
# Drop COD_PRO_RBD, NOM_COM_RBD, COD_DEPROV_RBD, NOM_DEPROV_RBD
estudiantes2018.drop(['COD_PRO_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD'], axis=1, inplace=True)


In [75]:
# Eliminar Unnamed
estudiantes2018 = estudiantes2018.drop(['Unnamed: 0'], axis=1)

In [76]:
# Eliminar AGNO, FEC_NAC_ALU de d2018
estudiantes2018 = estudiantes2018.drop(['AGNO', 'FEC_NAC_ALU'], axis=1)

In [77]:
# Drop NOM_COM_ALU, NOM_RBD y LET_CUR
estudiantes2018.drop(['NOM_RBD', 'NOM_COM_ALU', 'LET_CUR'], axis=1, inplace=True)

In [78]:
# Definir x e y
x = estudiantes2018.drop(['DESERTOR'], axis=1)
y = estudiantes2018['DESERTOR']


In [79]:
# Split train-test 80-20, stratify y random_state=2023
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

In [80]:
# Identificar las columnas categóricas
categorical_cols = x_train.select_dtypes(include=['object']).columns

# Realizar la codificación one-hot
x_train_encoded = pd.get_dummies(x_train, columns=categorical_cols)
x_test_encoded = pd.get_dummies(x_test, columns=categorical_cols)

# Entrenar el modelo con los datos codificados
rf_model = RandomForestClassifier()
rf_model.fit(x_train_encoded, y_train)

# Predecir con los datos codificados
y_pred = rf_model.predict(x_test_encoded)

In [81]:
# Entrenamos el random forest
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# Predecimos
y_pred = rf_model.predict(x_test)

# Evaluamos
print('Evaluamos')
print(classification_report(y_test, y_pred))

# Matriz de confusión
print('Matriz de confusión')
print(confusion_matrix(y_test, y_pred))

# Importancia de las variables
importances = list(rf_model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(x_train.columns, importances)]
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
[print('Variable: {:20} Importancia: {}'.format(*pair)) for pair in feature_importances];

# Guardar modelo
import pickle
pickle.dump(rf_model, open('data/modelos/rf_model.pkl', 'wb'))

# Cargar modelo
# rf_model = pickle.load(open('data/modelos/rf_model.pkl', 'rb'))



Evaluamos
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     15901

    accuracy                           1.00     15901
   macro avg       1.00      1.00      1.00     15901
weighted avg       1.00      1.00      1.00     15901

Matriz de confusión
[[15901]]
Variable: RBD                  Importancia: 0.0
Variable: DGV_RBD              Importancia: 0.0
Variable: COD_REG_RBD          Importancia: 0.0
Variable: COD_COM_RBD          Importancia: 0.0
Variable: COD_DEPE             Importancia: 0.0
Variable: COD_DEPE2            Importancia: 0.0
Variable: RURAL_RBD            Importancia: 0.0
Variable: ESTADO_ESTAB         Importancia: 0.0
Variable: COD_ENSE             Importancia: 0.0
Variable: COD_ENSE2            Importancia: 0.0
Variable: COD_ENSE3            Importancia: 0.0
Variable: COD_GRADO            Importancia: 0.0
Variable: COD_GRADO2           Importancia: 0.0
Variable: COD_JOR              Importancia: 0.0
Variable: COD_T

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 15, 20]
min_samples_leaf = [1, 2, 4, 6, 8]
bootstrap = [True, False]

# Crear el diccionario con los hiperparámetros
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

# Crear el modelo base
rf_model = RandomForestClassifier()

# Instanciar el grid search
grid_search = GridSearchCV(estimator=rf_model, param_grid=hyperparameter_grid, cv=5, n_jobs=-1, verbose=1)

# Entrenar el grid search
grid_search.fit(x_train_encoded, y_train)

# Mejores hiperparámetros
grid_search.best_params_

# Mejor modelo
best_model = grid_search.best_estimator_

# Predecir con el mejor modelo
y_pred = best_model.predict(x_test_encoded)

# Evaluamos
print('Evaluamos')
print(classification_report(y_test, y_pred))

# Matriz de confusión
print('Matriz de confusión')
print(confusion_matrix(y_test, y_pred)) 

# Importancia de las variables
importances = list(best_model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(x_train.columns, importances)]
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
[print('Variable: {:20} Importancia: {}'.format(*pair)) for pair in feature_importances]

# Guardar modelo
pickle.dump(best_model, open('data/modelos/best_model_rf.pkl', 'wb'))

In [ ]:
import shap

In [ ]:
explainer_object = shap.TreeExplainer(best_model)
shapley_values = explainer_object.shap_values(x_train_encoded)

i = 1

shap.initjs()
shap.force_plot(explainer_object.expected_value, shapley_values[i], features=x_train_encoded.loc[i], feature_names=x_train_encoded.columns)

In [ ]:
i = 2

shap.force_plot(explainer_object.expected_value, shapley_values[i], features=x_train_encoded.loc[i], feature_names=x_train_encoded.columns)

In [ ]:
shap_values = explainer_object(x_train_encoded)

In [ ]:
display(shap_values)

In [ ]:
exp = shap.Explainer(best_model, x_train_encoded)
shap_values = exp(x_train_encoded)

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.summary_plot(shapley_values, features=x_train_encoded, feature_names=x_train_encoded.columns)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
import lime
from lime import lime_tabular

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(x_train),
    feature_names=x_train.columns,
    class_names=[0, 1],
    mode='classification'
)

In [ ]:
i=12

exp_rf = explainer.explain_instance(
    data_row = test_X.iloc[i], 
    predict_fn = best_model.predict_proba
)

exp_rf.show_in_notebook(show_table=True)

In [ ]:
# mlp_model
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(hidden_layer_sizes= (150,120,100,50),
                          activation='relu',
                          max_iter= 100,
                          solver='sgd',
                          random_state=2023)
mlp_model.fit(x_train, y_train)

# Predecimos
y_pred = mlp_model.predict(x_test)

# Evaluamos
print('Evaluamos')
print(classification_report(y_test, y_pred))

# Matriz de confusión
print('Matriz de confusión')
print(confusion_matrix(y_test, y_pred))

# Guardar modelo
pickle.dump(mlp_model, open('data/modelos/mlp_model.pkl', 'wb'))

# Cargar modelo
# mlp_model = pickle.load(open('data/modelos/mlp_model.pkl', 'rb'))



Evaluamos
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     15901

    accuracy                           1.00     15901
   macro avg       1.00      1.00      1.00     15901
weighted avg       1.00      1.00      1.00     15901

Matriz de confusión
[[15901]]
